# Data description

## Resourses

* [Dataset resourse](https://www.kaggle.com/datasets/nikitarom/planets-dataset)
* [Full descriptions of data](https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/data)


## File formats

* **test-jpg** - folder with jpeg tiles for test model and validate it
  
* **train-jpg** - folder with jpeg tiles for train model

* **train_classes.csv** - a list of training file names and their labels, the labels are space-delimited

* **sample_submission.csv** - correct format of submission, contains all the files in the test set. For more information about the submission file, please go to the Evaluation page



Data are set of JPG chips for reference and practice (256x256 pixels, ~947.2m x 947.2m = 897187.84 m^2 = 89.7187 Hectares = 221.7 Acres). These chips were processed using the Planet visual product processor from tiff files and then saved as jpg chips. These chips are provided as a reference to the scene content.


## Class Labels

The class labels for this task were chosen in collaboration with Planet's Impact team and represent a reasonable subset of phenomena of interest in the Amazon basin. The labels can broadly be broken into three groups: **atmospheric conditions, common land cover/land use phenomena, and rare land cover/land use phenomena**. Each chip will have one and potentially more than one atmospheric label and zero or more common and rare labels. Chips that are labeled as cloudy should have no other labels, but there may be labeling errors.


As discussed in the data collection portion of this document, the chip labels are inherently noisy due to the labeling process and ambiguity of features, and scenes may either omit class labels or have incorrect class labels. 


* **Cloud Cover Labels**: clear, partly cloudy, cloudy, and haze

* **More Common Labels**:  rainforest, agriculture, rivers, towns/cities, and roads

* **Primary Rain Forest**: The overwhelming majority of the data set is labeled as "primary", which is shorthand for primary rainforest, or what is known colloquially as virgin forest. Generally speaking, the "primary" label was used for any area that exhibited dense tree cover

* **Habitation**: The habitation class label was used for chips that appeared to contain human homes or buildings. This includes anything from dense urban centers to rural villages along the banks of rivers. Small, single-dwelling habitations are often difficult to spot but usually appear as clumps of a few pixels that are bright white.

* **Cultivation**: Shifting cultivation is a subset of agriculture that is very easy to see from space, and occurs in rural areas where individuals and families maintain farm plots for subsistence.

* **Bare Ground**: Bare ground is a catch-all term used for naturally occuring tree free areas that aren't the result of human activity. Some of these areas occur naturally in the Amazon, while others may be the result from the source scenes containing small regions of biome much similar to the pantanal or cerrado.

* **Slash and Burn**: Slash-and-burn agriculture can be considered to be a subset of the shifting cultivation label and is used for areas that demonstrate recent burn events.

* **Selective Logging**: The selective logging label is used to cover the practice of selectively removing high value tree species from the rainforest (such as teak and mahogany). From space this appears as winding dirt roads adjacent to bare brown patches in otherwise primary rain forest.

* **Blooming**: Blooming is a natural phenomenon found in the Amazon where particular species of flowering trees bloom, fruit, and flower at the same time to maximize the chances of cross pollination. These trees are quite large and these events can be seen from space.

* **Conventional Mining**: There are a number of large conventional mines in the Amazon basin and the number is steadily growning. This label is used to classify large-scale legal mining operations.

* **"Artisinal" Mining**: Artisinal mining is a catch-all term for small scale mining operations. Throughout the Amazon, especially at the foothills of the Andes, gold deposits lace the deep, clay soils. Artisanal miners, sometimes working illegally in land designated for conservation, slash through the forest and excavate deep pits near rivers. They pump a mud-water slurry into the river banks, blasting them away so that they can be processed further with mercury - which is used to separate out the gold. The denuded moonscape left behind takes centuries to recover.

* **Blow Down**: Blow down, also called windthrow, is a naturally occurring phenomenon in the Amazon. Briefly, blow down events occur during microbursts where cold dry air from the Andes settles on top of warm moist air in the rainforest. The colder air punches a hole in the moist warm layer, and sinks down with incredible force and high speed (in excess of 100MPH). These high winds topple the larger rainforest trees, and the resulting open areas are visible from space. The open areas do not stay visible for along as plants in the understory rush in to take advantage of the sunlight.



In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.simplefilter('ignore')

import gc

from os import path
import sys
sys.path.append(path.abspath('..'))

In [ ]:
import os
import cv2
import torch
import pandas as pd
from PIL import Image
from pprint import pprint
import albumentations as albu
from albumentations.pytorch import ToTensorV2

from src.lightning_module import PlanetsModule

In [ ]:
DATA_FOLDER = '../dataset/planet/planet/'
!ls {DATA_FOLDER}

In [ ]:
df = pd.read_csv(os.path.join(DATA_FOLDER, 'train_classes.csv'))
df.head()

In [ ]:
df.shape

In [ ]:
total_tags = []
for i in range(df.shape[0]):
    for n in df['tags'][i].split(' '):
        if not n in total_tags:
            total_tags.append(n)
total_tags

In [ ]:
new_df_dict = {
    'Id': []
}
for tag in total_tags:
    new_df_dict[tag] = []

for i in range(df.shape[0]):
    new_df_dict['Id'].append(df['image_name'][i])
    for n in total_tags:
        if n in df['tags'][i]:
            new_df_dict[n].append(1)
        else:
            new_df_dict[n].append(0)

new_df = df.from_dict(new_df_dict)
new_df.head()

In [ ]:
new_df.shape

In [ ]:
len(total_tags)

In [ ]:
new_df.to_csv(
    os.path.join(DATA_FOLDER, 'df_train_ohe.csv'), 
    index=False
)